In [ ]:
import sys
import os

# Check if tkinter is available
try:
    import tkinter as tk
    from tkinter import ttk, messagebox
except ModuleNotFoundError as e:
    print("=" * 70)
    print("ERROR: tkinter is not installed!")
    print("=" * 70)
    print("\ntkinter is required for the GUI application.")
    print("\nTo install tkinter on macOS:")
    print("  1. Using Homebrew:")
    print("     brew install python-tk@3.11")
    print("     # or for Python 3.12+:")
    print("     brew install python-tk@3.12")
    print("\n  2. Or reinstall Python with tkinter support:")
    print("     pyenv install 3.11.x  # includes tkinter")
    print("\nTo install tkinter on Ubuntu/Debian:")
    print("  sudo apt-get install python3-tk")
    print("\nTo install tkinter on Fedora/RHEL:")
    print("  sudo dnf install python3-tkinter")
    print("\nAfter installation, restart the kernel and run this notebook again.")
    print("=" * 70)
    sys.exit(1)

import numpy as np
import pandas as pd
import joblib

PROJECT_HOME = os.getcwd()

# Check if model files exist
required_files = [
    PROJECT_HOME + "/models/best_xgb.pkl",
    PROJECT_HOME + "/models/best_lgb.pkl",
    PROJECT_HOME + "/models/best_cat.pkl",
    PROJECT_HOME + "/models/label_encoder.pkl"
]

missing_files = [f for f in required_files if not os.path.exists(f)]
if missing_files:
    print("=" * 70)
    print("ERROR: Required model files not found!")
    print("=" * 70)
    print("\nMissing files:")
    for f in missing_files:
        print(f"  - {f}")
    print("\nPlease run OptimalFertilizer_Approach1.ipynb first to generate the model files.")
    print("=" * 70)
    sys.exit(1)

# Load models
xgb = joblib.load(PROJECT_HOME+"/models/best_xgb.pkl")
lgb = joblib.load(PROJECT_HOME+"/models/best_lgb.pkl")
cat = joblib.load(PROJECT_HOME+"/models/best_cat.pkl")
le = joblib.load(PROJECT_HOME+"/models/label_encoder.pkl")


# -------------------------------
# Predict Function
# -------------------------------
def predict_fertilizer():

    try:
        # Read numeric inputs
        temp = float(entry_temp.get())
        hum = float(entry_hum.get())
        moist = float(entry_moist.get())
        nitrogen = float(entry_n.get())
        potass = float(entry_pot.get())
        phosph = float(entry_phos.get())

        # Read categorical dropdowns
        soil = soil_var.get()
        crop = crop_var.get()

        # Build dataframe for model
        X = pd.DataFrame([{
            "Soil Type": soil,
            "Crop Type": crop,
            "Temparature": temp,
            "Humidity": hum,
            "Moisture": moist,
            "Nitrogen": nitrogen,
            "Potassium": potass,
            "Phosphorous": phosph
        }])

        # Model predictions
        px = xgb.predict_proba(X)
        pl = lgb.predict_proba(X)
        pc = cat.predict_proba(X)

        # Weighted ensemble
        ensemble = 0.4 * pc + 0.35 * px + 0.25 * pl

        # Top-3 predictions
        idx = np.argsort(-ensemble, axis=1)[:, :3][0]
        labels = le.inverse_transform(idx)

        #result = f"Top-3 Recommended Fertilizers:\n1. {labels[0]}\n2. {labels[1]}\n3. {labels[2]}"
        result = f"Recommended Fertilizer: {labels[0]}" #\n2. {labels[1]}\n3. {labels[2]}"
        output_label.config(text=result)

    except Exception as e:
        messagebox.showerror("Error", str(e))


def validate_range(text, min_val, max_val):
    if text == "":      # allow empty while typing
        return True
    try:
        value = float(text)
        return min_val <= value <= max_val
    except ValueError:
        return False


def add_ranged_entry(frame, label, row, min_val, max_val):
    ttk.Label(frame, text=f"{label} ({min_val}–{max_val})").grid(row=row, column=0, padx=5, pady=5)

    vcmd = (frame.register(lambda t: validate_range(t, min_val, max_val)), "%P")

    entry = ttk.Entry(frame, validate="key", validatecommand=vcmd)
    entry.grid(row=row, column=1, padx=5, pady=5)
    return entry

# -------------------------------
# GUI Layout
# -------------------------------
root = tk.Tk()
root.title("Fertilizer Recommendation")

frame = ttk.Frame(root, padding=20)
frame.grid()

# Numeric inputs

entry_temp = add_ranged_entry(frame, "Temperature", 0, 0, 60)
entry_hum  = add_ranged_entry(frame, "Humidity", 1, 0, 100)
entry_moist = add_ranged_entry(frame, "Moisture", 2, 0, 100)
entry_n = add_ranged_entry(frame, "Nitrogen", 3, 0, 150)
entry_pot = add_ranged_entry(frame, "Potassium", 4, 0, 150)
entry_phos = add_ranged_entry(frame, "Phosphorous", 5, 0, 150)


# Categorical inputs
ttk.Label(frame, text="Soil Type").grid(row=6, column=0)
soil_var = tk.StringVar()
soil_menu = ttk.Combobox(frame, textvariable=soil_var,
                         values=["Sandy", "Clay", "Loamy", "Black", "Red"])
soil_menu.grid(row=6, column=1)

ttk.Label(frame, text="Crop Type").grid(row=7, column=0)
crop_var = tk.StringVar()
crop_menu = ttk.Combobox(frame, textvariable=crop_var,
                         values=["Barley", "Wheat", "Maize", "Cotton", "Sugarcane","Millets","Ground Nut","Tobacco"])
crop_menu.grid(row=7, column=1)

# Predict button
predict_btn = ttk.Button(frame, text="Predict Fertilizer", command=predict_fertilizer)
predict_btn.grid(row=8, column=0, columnspan=2, pady=10)

# Output box
output_label = ttk.Label(frame, text="  ", font=("Arial", 12))
output_label.grid(row=9, column=0, columnspan=2, pady=10)

root.mainloop()


/opt/homebrew/Cellar/python@3.11/3.11.14_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/pickle.py:1718: UserWarning: [15:37:09] WARNING: /Users/runner/work/xgboost/xgboost/src/gbm/../common/error_msg.h:83: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)
